# Learning from Unlabeled Data: Rise of Semi-Supervised and Self-Supervised Learning

## Technical requirements

We will use the following as technical requirements to run the code in this chapter:
- Python 3
- pip
- Tensorflow (with CUDA if you want to train models on GPUs)
    - Keras is installed as a dependency to this
- scikit-learn Python library
    - Numpy is installed as a dependency to this
- Jupyter notebook if running the code directly from Jupyter

In [2]:
! python3 -m pip install --upgrade pip

### For M1+ Macbook (64-bit ARM Based processor)

In [ ]:
! arch -arm64 pip3 install --upgrade pip
! arch -arm64 pip3 install tensorflow
! arch -arm64 pip3 install -U scikit-learn

### For Other Computer Systems

In [3]:
! pip3 install --upgrade pip
! pip3 install tensorflow
! pip3 install -U scikit-learn

  Using cached tensorflow-2.16.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (4.1 kB)
Using cached tensorflow-2.16.1-cp310-cp310-macosx_12_0_arm64.whl (227.0 MB)
  Using cached scikit_learn-1.4.1.post1-cp310-cp310-macosx_12_0_arm64.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.4.1.post1-cp310-cp310-macosx_12_0_arm64.whl (10.4 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 34.2 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.4.0-py3-none-any.whl (17 kB)


In [4]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.cluster import KMeans
import numpy as np

## 1. Introduction to Machine Learning

### 1.1 Building a Machine Learning Model

We usually divide the given data into 2 subsets - one for training and other for testing. Below we will use CIFAR-10 dataset to build a classifier model and a clustering model. You can see we first load the train and test images and labels and then normalize each image to be in range [-1, 1] before it could be fed into our ML model for training and testing.

In [5]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images = train_images / 127.5 - 1
test_images = test_images / 127.5 - 1

Next, we need to decide on the right learning method and algorithm that would solve the problem at hand. If the problem is to predict classes for each test image, we would train the model using labels of each image through a convolutional neural network as described below.

In [6]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Next, we would select an appropriate loss function and optimization technique.

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Finally, we train the model, validate and test it using the right evaluation metrics.

In [8]:
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.4111 - loss: 1.6087 - val_accuracy: 0.5973 - val_loss: 1.1089
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.6343 - loss: 1.0351 - val_accuracy: 0.6780 - val_loss: 0.9265
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.7032 - loss: 0.8501 - val_accuracy: 0.7014 - val_loss: 0.8589
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.7402 - loss: 0.7432 - val_accuracy: 0.7215 - val_loss: 0.8004
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7698 - loss: 0.6629 - val_accuracy: 0.7259 - val_loss: 0.8055
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.7948 - loss: 0.5904 - val_accuracy: 0.7385 - val_loss: 0.7672
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.8160 - loss: 0.5343 - val_accuracy: 0.7362 - val_loss: 0.8105
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.8324 -

We'll get the output as follows for each epoch and final test accuracy.

But if the problem is to create clusters of images that represent the same group of entities, we would not use any labels for that purpose. So we collate training and test images and flatten them so that we can use a K-means clustering algortihm to get image clusters.


In [9]:
images = np.concatenate((train_images, test_images))
images_flattened = images.reshape(images.shape[0], -1)

kmeans = KMeans(n_clusters=10, random_state=42)
cluster_assignments = kmeans.fit_predict(images_flattened)

print(cluster_assignments[:100])

[1 0 3 6 3 5 4 4 9 5 6 2 3 6 5 7 7 6 2 1 8 4 6 5 5 3 5 6 4 2 0 3 5 1 4 4 9
 5 3 0 3 8 4 2 0 5 0 0 8 8 1 0 1 2 6 7 1 5 5 3 0 9 0 5 0 9 4 7 5 3 4 5 2 5
 4 4 7 0 0 8 4 4 0 3 1 6 1 1 5 4 4 5 9 0 3 2 0 5 5 6]
